---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple
from random import random

In [2]:
@dataclass(frozen=True, order=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

# Nodes and Edges

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the attributes source and destination, but bidirectional edge cannot have these attributes. Which node should be a bidirectional edge's "source"? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional ones either. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we *can* build undirected graph with directional edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird and I want to avoid this.

`BaseEdge` is an abstract base class for edges. An edge has two nodes (`.node1` and `.node2`), can return them in a tuple (`.nodes`) and can be represented by a string where names of both nodes are separated by a separator. `BaseEdge` is also hashable.

`BaseEdge` also has `._separator` which is an abstract property to be determined by a concrete subclass. Its implementation depends on whether the edge is bidirectional or unidirectional.

(@ property means accessing `.nodes` on an instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.)

In [3]:
@dataclass(frozen=True, order=True)
class BaseEdge(ABC):
    """Abstract base class for immutable graph edge."""

    node1: Node
    node2: Node
    weight: int | float | None = None

    def __post_init__(self):
        # if weighted, then weight not None + bool operators lazy so this works
        if self.weighted and self.weight < 0:  # type: ignore
            raise ValueError("Weight cannot be negative")

    @property
    def weighted(self) -> bool:
        return self.weight is not None

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self.node1, self.node2

    @property
    @abstractmethod
    def _separator(self) -> Tuple[str, str]:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        left, right = self._separator
        if self.weighted:
            return f"{node1} {left}({self.weight}){right} {node2}"
        else:
            return f"{node1} {left}{right} {node2}"


@dataclass(frozen=True, order=True)
class Edge(BaseEdge):
    """Bidirectional immutable graph edge."""

    def __post_init__(self):
        # Canonical ordering for dataclass's __eq__ and __hash__
        # to work correctly and for better (consistent) str representation
        if self.node1 > self.node2:
            node1 = self.node1
            object.__setattr__(self, "node1", self.node2)
            object.__setattr__(self, "node2", node1)
        super().__post_init__()

    @property
    def _separator(self) -> Tuple[str, str]:
        return "<--", "-->"


@dataclass(frozen=True, order=True)
class DirectedEdge(BaseEdge):
    """Unidirectional immutable graph edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float | None = None):
        # Improves signature of the constructor to reflect src and dest
        # Invariant: src=node1 and dest=node2
        super().__init__(src, dest, weight)

    def __repr__(self):
        # Reflects the renaming of node1 and node2 to src and dest
        return super().__repr__().replace("node1", "src").replace("node2", "dest")

    @property
    def src(self) -> Node:
        return self.nodes[0]

    @property
    def dest(self) -> Node:
        return self.nodes[1]

    @property
    def _separator(self) -> Tuple[str, str]:
        return "---", "-->"

## Testing of Nodes and Edges

In [5]:
node1 = Node("1")
node2 = Node("2")

# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'

e12 = Edge(node1, node2)
we12_314 = Edge(node1, node2, 3.14)
dir12 = DirectedEdge(node1, node2)
wdir12_314 = DirectedEdge(node1, node2, 3.14)

e21 = Edge(node2, node1)
we21_314 = Edge(node2, node1, 3.14)
dir21 = DirectedEdge(node2, node1)
wdir21_314 = DirectedEdge(node2, node1, 3.14)

we12_3 = Edge(node1, node2, 3)
wdir12_3 = DirectedEdge(node1, node2, 3)

In [6]:
print(f"{e12 == e21}: {e12} equals {e21}")
print(f"{we12_314 == we21_314}: {we12_314} equals {we21_314}")
print(f"{dir12 == dir21}: {dir12} equals {dir21}")
print(f"{wdir12_314 == wdir21_314}: {wdir12_314} equals {wdir21_314}")
print(f"{we12_314 == we12_3}: {we12_314} equals {we12_3}")
print(f"{wdir12_314 == wdir12_3}: {wdir12_314} equals {wdir12_3}")

True: 1 <----> 2 equals 1 <----> 2
True: 1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2
False: 1 -----> 2 equals 2 -----> 1
False: 1 ---(3.14)--> 2 equals 2 ---(3.14)--> 1
False: 1 <--(3.14)--> 2 equals 1 <--(3)--> 2
False: 1 ---(3.14)--> 2 equals 1 ---(3)--> 2


In [7]:
print({e12, e21})  # exactly what we wanted because they are the same edge!
print({we12_314, we21_314})
print({node1, Node("1"), node2})

{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)}
{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=3.14)}
{Node(name='2'), Node(name='1')}


In [8]:
print(repr(DirectedEdge(node1, node2)))
print(repr(Edge(node1, node2)))

DirectedEdge(src=Node(name='1'), dest=Node(name='2'), weight=None)
Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)


In [9]:
try:
    Edge(node1, node2, -3.14)
except ValueError:
    print("test passed")

try:
    DirectedEdge(node1, node2, -3.14)
except ValueError:
    print("test passed")

test passed
test passed


# Digraphs and Graphs

In [ ]:
class Graph:
    """Graph."""

    # _nodes and _edges are set of the nodes, edges in the graph, respectively.
    # _adjacency is a dict mapping each node to its children implemented as
    # dict, where the key is the child node and the value is set of edges.
    # (that allows for two or more different edges between two nodes)
    # Internally, an "edge" is one-way only.
    # Bidirectional edges treated as having an "edge" in both directions.

    def __init__(self, directed=False, weighted=False):
        self._nodes = set()
        self._edges = set()
        self._adjacency = dict()
        self.directed = directed
        self.weighted = weighted

    def has(self, node_or_edge: Node | BaseEdge) -> bool:
        """Return bool whether node or edge in graph."""
        if isinstance(node_or_edge, Node):
            return node_or_edge in self._nodes
        else:  # python's typing ensures obj *either* Node or BaseEdge
            return node_or_edge in self._edges

    def _check_exists(self, node_or_edge: Node | BaseEdge) -> None:
        """Raises ValueError if node or edge not in graph."""
        obj = "node" if isinstance(node_or_edge, Node) else "edge"
        if not self.has(node_or_edge):
            raise ValueError(f"{obj} not in graph")

    def get_nodes(self, sort=False) -> list[Node]:
        """Get list of nodes, possibly sorted."""
        if sort:
            return sorted(self._nodes)
        else:
            return list(self._nodes)

    def add_edge(self, edge: DirectedEdge | Edge) -> None:
        """
        Add edge.

        Directedness and weightedness of graph and edge must match.
        Undirected edges treated as two directed edges.
        """
        node1, node2 = edge.nodes
        # Check both endpoint nodes in graph
        self._check_exists(node1)
        self._check_exists(node2)
        # Check directedness and weightedness
        # different Errors raised because directedness implemented as a subclass
        # but weightedness implemented as a flag
        if isinstance(edge, DirectedEdge) and not self.directed:
            raise TypeError("This graph is undirected, cannot add directed edge")
        if edge.weighted and not self.weighted:
            raise ValueError("This graph is unweighted, cannot add weighted edge")
        if self.weighted and not edge.weighted:
            raise ValueError("This graph is weighted, cannot add unweighted edge")
        # Check uniqueness of edge and actually add the edge
        # self.has(edge) cannot be used to check uniqueness because when directed
        # graph has one edge between two nodes, then adding undirected edge between
        # same nodes not caught as duplicity in one direction
        if isinstance(edge, DirectedEdge):
            try:
                if edge in self._adjacency[node1][node2]:
                    raise ValueError("Duplicate edge")
                self._adjacency[node1][node2].add(edge)
            except KeyError:  # must be [node2], since [node1] added when node added
                self._adjacency[node1][node2] = {edge}

        else:  # isinstance(edge, Edge)
            try:
                # THIS DOESNT CATCH DUPLICITY BECAUSE edge CAN BE DIFFERENT FROM WHAT IS STORED YET IT CONTAINS IT
                # SOLVE THIS BY IMPLEMENTING SUBSET CHECKING? <=?
                in12 = edge in self._adjacency[node1][node2]
            except KeyError:
                in12 = False
            try:
                in21 = edge in self._adjacency[node2][node1]
            except KeyError:
                in21 = False

            if in12 or in21:
                msg = "Adding this undirected node would create duplicity in at least one direction"
                raise ValueError(msg)

            try:
                self._adjacency[node1][node2].add(edge)
            except KeyError:
                self._adjacency[node1][node2] = {edge}
            try:
                self._adjacency[node2][node1].add(edge)
            except KeyError:
                self._adjacency[node2][node1] = {edge}
        self._edges.add(edge)

    def get_edges(self, sort=False) -> list[BaseEdge]:
        """Get list of edges, possibly sorted."""
        if sort:
            return sorted(self._edges)
        else:
            return list(self._edges)

    def add_node(self, node: Node) -> None:
        """Add node to the graph."""
        if self.has(node):
            raise ValueError("Duplicate node")
        else:
            self._nodes.add(node)
            self._adjacency[node] = dict()

    def children_of(self, node: Node) -> list[Node]:
        """Return list of child nodes of node."""
        self._check_exists(node)
        return list(self._adjacency[node])  # returns list of keys

    def __str__(self) -> str:
        # DOEST DEALT WITH DIRECTED GRAPHS CONTAINING BOTH EDGES AND DIRECTED EDGES, BECAUSE THESE TYPES
        # DONT HAVE < ORDERING AND CANNOT BE SORTED
        # could be cashed, but graphs large enough for cashing to
        # make sense wouldn't make sense to try to print anyway
        if len(self._nodes) == 0:
            return "Empty graph."
        else:
            to_print = []
            connected = set()
            edges = self.get_edges(sort=True)
            for edge in edges:
                connected |= {*edge.nodes}
            unconnected = list(self._nodes - connected)
            for edge in edges:
                to_print.append(f"{edge}")
            res = "\n".join(to_print)
            if len(unconnected) == 0:
                res += "\nNo unconnected nodes."
            else:
                res += "\nUnconnected nodes:\n"
                for node in unconnected:
                    res += f"{node}\n"
                res = res[:-1]
            return res

## Testing Digraphs and Graphs

In [11]:
edges = [(0, 1), (1, 2), (2, 3), (2, 4), (3, 4), (3, 5), (0, 2), (1, 0), (3, 1), (4, 0)]
nodes = []
for name in range(8):
    nodes.append(Node(str(name)))
digraph = Graph(directed=True, weighted=False)
digraph_w = Graph(directed=True, weighted=True)
graph = Graph(weighted=False)
graph_w = Graph(weighted=True)
for n in nodes:
    for g in [digraph, digraph_w, graph, graph_w]:
        g.add_node(n)
for n1, n2 in edges:
    n1, n2 = str(n1), str(n2)
    w = round(random(), 2)
    digraph.add_edge(DirectedEdge(Node(n1), Node(n2)))
    digraph_w.add_edge(DirectedEdge(Node(n1), Node(n2), weight=w))
    try:
        graph.add_edge(Edge(Node(n1), Node(n2)))
    except ValueError as m:
        pass
    try:
        graph_w.add_edge(Edge(Node(n1), Node(n2), weight=w))
    except ValueError as m:
        pass

In [12]:
print(digraph)

0 -----> 1
0 -----> 2
1 -----> 0
1 -----> 2
2 -----> 3
2 -----> 4
3 -----> 1
3 -----> 4
3 -----> 5
4 -----> 0
Unconnected nodes:
7
6


In [13]:
print(digraph_w)

0 ---(0.39)--> 1
0 ---(0.21)--> 2
1 ---(0.09)--> 0
1 ---(0.61)--> 2
2 ---(0.55)--> 3
2 ---(0.38)--> 4
3 ---(0.01)--> 1
3 ---(0.94)--> 4
3 ---(0.91)--> 5
4 ---(0.62)--> 0
Unconnected nodes:
7
6


In [14]:
print(graph)

0 <----> 1
0 <----> 2
0 <----> 4
1 <----> 2
1 <----> 3
2 <----> 3
2 <----> 4
3 <----> 4
3 <----> 5
Unconnected nodes:
7
6


In [15]:
print(graph_w)

0 <--(0.09)--> 1
0 <--(0.39)--> 1
0 <--(0.21)--> 2
0 <--(0.62)--> 4
1 <--(0.61)--> 2
1 <--(0.01)--> 3
2 <--(0.55)--> 3
2 <--(0.38)--> 4
3 <--(0.94)--> 4
3 <--(0.91)--> 5
Unconnected nodes:
7
6


In [16]:
print(Graph(weighted=False))

Empty graph.


In [17]:
try:
    graph_w.add_edge(Edge(Node("8"), Node("9")))
except ValueError as m:
    print(m)

node not in graph


In [18]:
try:
    graph.add_edge(DirectedEdge(Node("0"), Node("5")))
except TypeError as m:
    print(m)

This graph is undirected, cannot add directed edge


In [19]:
try:
    graph_w.add_edge(Edge(Node("0"), Node("5")))
except ValueError as m:
    print(m)

This graph is weighted, cannot add unweighted edge


In [20]:
try:
    digraph.add_edge(Edge(Node("0"), Node("5"), weight=0.5))
except ValueError as m:
    print(m)

This graph is unweighted, cannot add weighted edge


In [21]:
print(digraph)

0 -----> 1
0 -----> 2
1 -----> 0
1 -----> 2
2 -----> 3
2 -----> 4
3 -----> 1
3 -----> 4
3 -----> 5
4 -----> 0
Unconnected nodes:
7
6


In [22]:
try:
    digraph.add_edge(Edge(Node("0"), Node("1")))
except ValueError as m:
    print(m)

In [23]:
print(digraph)

TypeError: '<' not supported between instances of 'Edge' and 'DirectedEdge'